# MODEL TRAINING - DEMO

In [ ]:
#external libraries
import numpy as np
import os
import matplotlib.pyplot as plt
import matplotlib.colors as clt
import plotly
import plotly.subplots as sb
import plotly.express as px
import plotly.graph_objects as go
import dotenv
import pandas as pd
import scipy.fft as fft
import scipy.signal as sg
import scipy.io as sio
import pickle as pkl
import xgboost as xgb
import time
import uuid

#project library
from spinco import *

#environment variables
dotenv.load_dotenv('lab.env')

#project variables
demopath=os.getcwd()


## experiment id
we'll use it to create a folder at the end of the script

In [ ]:
experimentId=str(uuid.uuid4())

## define a fixed samplerate

In [ ]:
samplerate=200

## load dreams

In [ ]:
signals, annotations, signalsMetadata = loadDREAMSSpindlesDemo(demopath)

In [ ]:
annotations=annotations[annotations.type=='spindle'].reset_index(drop=True)
len(annotations)

In [ ]:
minDuration=0.3
maxDuration=5
annotations=annotations[annotations.duration>minDuration].reset_index(drop=True)
annotations=annotations[annotations.duration<maxDuration].reset_index(drop=True)
len(annotations)

In [ ]:
annotations.head()

In [ ]:
signalsMetadata.head()

## define a fixed feature selection to use

In [ ]:
featureSelection=loadPickle("featureSelection26.pkl")
featureSelection

## load data split
LOOCV using 5 subjects, 1 for validation

In [ ]:
#by now we used fixed CV data splits instead
dataSplits=loadPickle("dataSplits_LOOCV_5S.pkl")
dataSplits

## define annotation criteria

In [ ]:
annotationCriteria=pd.DataFrame({
    'criteriumId':['0001'],
    'criteriumName':['union'],
    'labelerIdList':[['0001','0002']]
})

In [ ]:
annotationCriteria

In [ ]:
experimentModels=pd.merge(annotationCriteria,dataSplits,how='cross')

In [ ]:
experimentModels

## model fitting

In [ ]:
n_jobs=4 # paralelization parameter for xgboost (keep it 8 as maximum)
learning_rate=0.4
subsample=0.6

num_boost_round=60  #number of boosting rounds per model

models=[]
modelIds=[]
valF1s=[]
spindleTimeRates=[]
for index, row in experimentModels.iterrows():
    print('**********************')
    print(str(index+1)+' of '+str(len(experimentModels)))
    #Define annotations criterium
    usedAnnotations=annotations[annotations.labelerId.isin(row.labelerIdList)].reset_index(drop=True)
    #Features
    trainFeatures=loadFeatureMatrix(row.train,featureSelection,signalsMetadata,samplerate,demopath)
    #Labels
    trainLabels=loadLabelsVector(row.train,usedAnnotations,signalsMetadata,samplerate)
    
    #Train the models
    params={
        'n_jobs':n_jobs,
        'learning_rate':learning_rate,
        'subsample':subsample,
        'objective':'binary:logistic'
    }
    trainDMatrix=xgb.DMatrix(data=trainFeatures,label=trainLabels)
    xgb_model = xgb.train(params,trainDMatrix,num_boost_round=num_boost_round)
    spindleTimeRates.append(sum(trainLabels)/len(trainLabels))
    models.append(xgb_model)
    modelIds.append(str(uuid.uuid4()))
    

In [ ]:
experimentModels["modelId"]=modelIds
experimentModels["spindleTimeRate"]=spindleTimeRates

In [ ]:
experimentModels

## save results

In [ ]:
#create parent folder
if not os.path.isdir(demopath+"/DREAMS/experiments/"):
    os.mkdir(demopath+"/DREAMS/experiments/")
experimentpath=demopath+"/DREAMS/experiments/"+experimentId
os.mkdir(experimentpath)
#save each of the models
for id, model in zip(modelIds,models):
    model.save_model(experimentpath+"/"+id+".json")
#save experiment information
dumpPickle(experimentpath+"/experimentModels.pkl",experimentModels)
dumpPickle(experimentpath+"/featureSelection.pkl",featureSelection)

# COPY THE UUID BELOW TO THE NEXT NOTEBOOK

In [ ]:
print(experimentId)